<a href="https://colab.research.google.com/github/yorkjong/stock-reports/blob/main/notebooks/ibd_reports.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stock Screening Using IBD's RS Rating Methodology

This notebook demonstrates stock screening techniques inspired by the methodology used in Investor's Business Daily (IBD). It incorporates the (price) RS Rating, EPS RS Rating, and Revenue RS Rating to evaluate stocks based on price performance, earnings, and revenue growth. The Price RS Rating typically reflects a stock’s performance over the past year, but a 3-month version is also available to assist with short-term trading decisions.

## Install and Setup (this section will be executed automatically)

### Install Required Packages

In [20]:
%pip install "git+https://github.com/yorkjong/vistock.git@feature/ranking_utils"
%pip install requests-cache

!wget -O NotoSansTC-Regular.ttf https://share.cole.tw/d/Tools%20-%20MAC/Fonts/Noto_Sans_TC/static/NotoSansTC-Regular.ttf?sign=bATsZP5QZdI_2EM15sAbcAE48Cacle91CpwUNOCMuM8=:0

  Cloning https://github.com/yorkjong/vistock.git (to revision feature/ranking_utils) to /tmp/pip-req-build-slfdbghr
  Running command git clone --filter=blob:none --quiet https://github.com/yorkjong/vistock.git /tmp/pip-req-build-slfdbghr
  Running command git checkout -b feature/ranking_utils --track origin/feature/ranking_utils
  Switched to a new branch 'feature/ranking_utils'
  Branch 'feature/ranking_utils' set up to track remote branch 'feature/ranking_utils' from 'origin'.
  Resolved https://github.com/yorkjong/vistock.git to commit f7f08948581269857ec6cff2f9e27922231b299d
  Preparing metadata (setup.py) ... done
--2024-10-11 12:31:23--  https://share.cole.tw/d/Tools%20-%20MAC/Fonts/Noto_Sans_TC/static/NotoSansTC-Regular.ttf?sign=bATsZP5QZdI_2EM15sAbcAE48Cacle91CpwUNOCMuM8=:0
Resolving share.cole.tw (share.cole.tw)... 180.218.0.193
Connecting to share.cole.tw (share.cole.tw)|180.218.0.193|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7110560 (6.8M) [

### Setup and Configuration

In [21]:
# @title Enable Requests Cache
import requests_cache
requests_cache.install_cache('ibd_cache', expire_after=3600)

In [22]:
# @title Set Chinese Font for matplotlib
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt

fm.fontManager.addfont('NotoSansTC-Regular.ttf')
font_name = 'Noto Sans TC'
if font_name not in plt.rcParams['font.sans-serif']:
    plt.rcParams['font.sans-serif'].insert(0, font_name)

In [23]:
# @title ReadOnlyGitHub
import requests
import pandas as pd
from io import StringIO

class ReadOnlyGitHub:
    def __init__(self, repo_owner, repo_name, dir='', branch='main'):
        dir = dir.strip('/')
        base = 'https://raw.githubusercontent.com'
        if dir:
            self.raw_url = f'{base}/{repo_owner}/{repo_name}/{branch}/{dir}'
        else:
            self.raw_url = f'{base}/{repo_owner}/{repo_name}/{branch}'

        base = 'https://api.github.com/repos'
        if dir:
            self.api_url = f'{base}/{repo_owner}/{repo_name}/contents/{dir}'
        else:
            self.api_url = f'{base}/{repo_owner}/{repo_name}/contents'
        self.branch = branch

    def file_exists(self, file_path):
        url = f'{self.raw_url}/{file_path}'
        response = requests.head(url)
        return response.status_code == 200

    def list_filenames(self, dir_path=''):
        if dir_path:
            url = f'{self.api_url}/{dir_path}?ref={self.branch}'
        else:
            url = f'{self.api_url}?ref={self.branch}'
        response = requests.get(url)
        if response.status_code == 200:
            files = response.json()
            return [item['name'] for item in files]
        elif response.status_code == 404:
            print(f"Directory not found: {url}")
            return []
        else:
            print(f"Request failed: {response.status_code} - {response.text}")
            return []

    def download_csv(self, file_path):
        url = f'{self.raw_url}/{file_path}'
        if self.file_exists(file_path):
            return pd.read_csv(url)
        else:
            return pd.DataFrame()

# Create a GitHub instance
github = ReadOnlyGitHub(
    repo_owner='yorkjong',
    repo_name='stock-reports',
    dir='ibd',
    branch='data'
)

In [24]:
# @title Taiwan Stock Name Lookup

class StockNameLookup:
    _df = None  # Class-level variable to hold the DataFrame

    @classmethod
    def _load_data(cls):
        if cls._df is None:  # Check if the DataFrame is already loaded
            gh = ReadOnlyGitHub(
                repo_owner='yorkjong',
                repo_name='stock-reports',
                dir='data/stock_list',
            )
            cls._df = gh.download_csv('taiwan_stock_OpenAPI.csv')

    @classmethod
    def tw_stock_name(cls, ticker):
        cls._load_data()  # Ensure data is loaded before accessing

        code = ticker.split('.')[0]  # Extract the code part

        # Filter the DataFrame to find the stock name for the given code
        stock_name = cls._df.loc[cls._df['Code'] == code, 'Name']

        # Check if the stock_name is empty and return an appropriate message
        if not stock_name.empty:
            return stock_name.values[0]  # Return the first matched stock name
        else:
            return None  # Return None if ticker not found


def tw_stock_name(ticker):
    return StockNameLookup.tw_stock_name(ticker)

In [25]:
# @title Metadata

def parse_metadata(filename):
    components = filename.split('_')
    if len(components) != 5:
        raise ValueError("Filename does not have the expected number of components")

    source, kind, period, rs_win, date = components
    rs_win = rs_win.replace('ibd', '')
    date = date.replace('.csv', '')

    return {
        "Source": source,
        "Type": kind,
        "Period": period,
        "RS window": rs_win,
        "Date": date
    }


def print_metadata(meatadata):
    for key, value in meatadata.items():
        print(f"{key}: {value}")

In [26]:
# @title IBD Financial files
import re
from datetime import datetime


def get_latest_file(file_list, source):
    pattern = rf'^{re.escape(source)}_(\w+)_fin_(\d+)\.csv$'
    matching_files = []

    for file in file_list:
        match = re.match(pattern, file)
        if match:
            date_str = match.group(2)
            date = datetime.strptime(date_str, "%Y%m%d")
            matching_files.append((file, date))

    if not matching_files:
        return None

    return max(matching_files, key=lambda x: x[1])[0]


def fin_download_latest_csv(source):
    gh = ReadOnlyGitHub(
        repo_owner='yorkjong',
        repo_name='stock-reports',
        dir='ibd_fin',
        branch='data'
    )
    file_list = gh.list_filenames()
    fin_filename = get_latest_file(file_list, source)
    df = gh.download_csv(fin_filename)
    return df, fin_filename

In [27]:
# @title DataFrame Utilities

def print_column(df, column):
    if column in df.columns:
        print(', '.join(df[column]))

In [28]:
# @title Source of Tickers

def tickers_from_df(df):
    if 'Name' in df.columns:
        return [name.strip() for names in df['Name']
                for name in names.split(',')]
    elif 'Ticker' in df.columns:
        return [ticker.strip() for tickers in df['Ticker']
                for ticker in tickers.split(',')]
    return []

def major_indices():
    return ['^DJI', '^IXIC', '^NDX', '^RUT', '^SOX',
            '^TWII', '^N225', '^HSI',
            '^STOXX50E', '^FTSE', '^GDAXI', '^FCHI', '^GSPTSE']

def sector_indices():
    return ['SOXX', 'DVY',
            'IWB','IWM', 'IWV',  'IJR',
            'ITB', 'IHI', 'IYC', 'ITA', 'IAK',
            'IYZ', 'IYT', 'IYR', 'IYF', 'IYJ',
            'IYG', 'IYH', 'IYK', 'IDU', 'IYE', 'IHE',
            'IAT', 'IAI', 'IEO', 'IYM', 'IHF']

In [29]:
# @title Checkboxes

import ipywidgets as widgets

def cbs_create(symbols, n_pre_checked=10):
    '''Create a list of checkboxes'''
    return [
        widgets.Checkbox(
            value=(i < n_pre_checked),  # Set first n items as checked
            description=symbol,
            layout=widgets.Layout(width='auto'),
            style={'description_width': 'auto'}
        )
        for i, symbol in enumerate(symbols)
    ]

def cbs_with_grid(checkboxes, n_cols=5):
    '''Create a grid layouting the given checkboxes'''
    return widgets.GridBox(checkboxes, layout=widgets.Layout(
        width='auto',
        grid_template_columns=f'repeat({n_cols}, 1fr)',
        grid_gap='10px'  # Add some space between the checkboxes
    ))

def cbs_get_selected(checkboxes):
    '''Get the selected symbols from the given checkboxes'''
    return [checkbox.description for checkbox in checkboxes if checkbox.value]

def cbs_unselect_all(checkboxes):
    '''Unselect all checkboxes in the given list'''
    for checkbox in checkboxes:
        checkbox.value = False

def cbs_select_top(checkboxes, n=10):
    '''Select the top n checkboxes in the given list'''
    for i, checkbox in enumerate(checkboxes):
        checkbox.value = (i < n)

In [30]:
# @title Dropdown Menus

def create_period_dropdown(value='2y'):
    return widgets.Dropdown(
        options=['1y', '2y', '5y'],
        value=value,
        description='Period:',
    )

def create_interval_dropdown(value='1wk'):
    return widgets.Dropdown(
        options=['1d', '1wk'],
        value=value,
        description='Interval:',
    )

def create_rs_window_dropdown(value='3mo'):
    return widgets.Dropdown(
        options=['3mo', '12mo'],
        value=value,
        description='rs_window:',
    )

def create_style_dropdown(desc=None, value=None):
    return widgets.Dropdown(
        options=['default', 'classic', 'yahoo', 'charles', 'tradingview', 'binance', 'binancedark', 'mike', 'nightclouds', 'checkers', 'ibd', 'sas', 'starsandstripes', 'kenan', 'blueskies', 'brasil'],
        value='yahoo' if value is None else value,
        description='Style:' if desc is None else desc,
        style={'description_width': 'initial'},
    )

def create_template_dropdown(desc=None, value=None):
    return widgets.Dropdown(
        options=['plotly', 'plotly_white', 'plotly_dark', 'ggplot2', 'seaborn', 'simple_white', 'presentation', 'xgridoff', 'ygridoff'],
        value='plotly_dark' if value is None else value,
        description='Template:' if desc is None else desc,
        style={'description_width': 'initial'},
    )

In [31]:
# @title Multiple Searchable Dropdown Menus
import ipywidgets as widgets

def create_search_box():
    '''Create a Text widget for search input'''
    return widgets.Text(
        description='Search:',
        placeholder='Type to search',
        layout=widgets.Layout(width='auto')
    )

def create_dropdown(options, description='Stock:'):
    '''Create a Dropdown widget for displaying filtered options'''
    return widgets.Dropdown(
        description=description,
        options=[None] + options,  # None as the default option
        layout=widgets.Layout(width='auto'),
        value=None  # Set default value to None
    )

def update_dropdown(change, dropdown, options):
    '''Update the options in the dropdown based on search input'''
    search_text = change['new'].lower()
    filtered = [option for option in options if search_text in option.lower()]
    if filtered:
        dropdown.options = [None] + filtered
        dropdown.value = filtered[0]  # Auto-select the first matching option
    else:
        dropdown.options = [None]  # Retain only the None option if no match

def remove_duplicates_preserve_order(lst):
    '''Remove duplicates from a list while preserving order'''
    seen = set()
    result = []
    for item in lst:
        if item and item not in seen:
            seen.add(item)
            result.append(item)
    return result

def get_dropdowns_selected_options(dropdowns):
    '''Get selected options from the dropdowns'''
    selected = [dropdown.value for dropdown in dropdowns if dropdown.value]
    return remove_duplicates_preserve_order(selected)

def create_search_dropdowns(options, max_selections):
    '''Create a layout with search boxes and dropdowns'''
    # Create UI components
    search_boxes = [create_search_box() for _ in range(max_selections)]
    dropdowns = [create_dropdown(options) for _ in range(max_selections)]

    # Bind search box and dropdown menu events
    for search_box, dropdown in zip(search_boxes, dropdowns):
        search_box.observe(lambda change, dropdown=dropdown:
                            update_dropdown(change, dropdown, options),
                            names='value')
    # Create the layout
    controls = [widgets.HBox([search_box, dropdown])
                for search_box, dropdown  in zip(search_boxes, dropdowns)]
    layout = widgets.VBox(controls)
    return dropdowns, layout

In [32]:
# @title Outputs
outputs = widgets.VBox()

In [33]:
# @title enable_plotly_in_cell
# ref. https://stackoverflow.com/questions/76593068/plotly-figure-not-rendering-in-ipywidgets-interact-function-google-colab
def enable_plotly_in_cell():
  import IPython
  from plotly.offline import init_notebook_mode
  display(IPython.core.display.HTML('''<script src="/static/components/requirejs/require.js"></script>'''))
  init_notebook_mode(connected=False)

### Glossary of Terms

#### Source (The source of stocks to analyze)
- This could include stocks traded on exchanges or components of a specific index.
- Common abbreviation(s) for the exchange or market sector:
  - For Taiwan Markets, possible values include:
    - **`TWSE`**: Taiwan Stock Exchange (台灣上市股票交易所)
    - **`TPEX`**: Taipei Exchange (上櫃交易所)
    - **`ESB`**: Emerging Stock Board (興櫃交易所)
  - Can also be combined with `+` (e.g., `TWSE+TPEX`, `TWSE+TPEX+ESB`).
  - For America Markets, possible values include:
    - **`SPX`**: S&P 500 (標普五百指數)
    - **`DJIA`**: Dow Jones Industrial Average (道瓊指數)
    - **`NDX`**: NASDAQ-100 (納斯達克一百指數)
    - **`SOX`**: PHLX Semiconductor Index (費半指數)
  - Multiple indices can be combined using `+` (e.g., `SPX+DJIA+NDX+SOX`).

#### Period (Historical Data Time Range)
- The time range for which to fetch historical data.
  - **`2y`**: 2 years
  - **`6mo`**: 6 months

#### RS Window (Window for RS Calculation)
- The time window ('3mo' or '12mo') for Relative Strength:
  - **`3mo`**: 3 months
  - **`12mo`**: 12 months

#### RS (Relative Strength)
- Relative Strength (RS) is a metric used to evaluate the performance of a stock relative to a benchmark index.
  - A higher RS rating indicates that the stock has outperformed the index, while a lower RS rating suggests underperformance.
  - A value of 100 represents the performance of the benchmark index or market.
- The IBD RS calculates the performance over the last year, with the most recent quarter weighted double.
- The IBD 3-month RS calculates the performance over the last quarter.

#### RS Rating
- RS Rating, ranging from 1 (worst) to 99 (best), evaluates the price performance of a stock relative to a benchmark index.

# Execute Actions Step by Step


## Step 1. Select and Preview a File

---



In [37]:
# @title Step 1.1 Select a File

import ipywidgets as widgets

with requests_cache.disabled():
    filenames = github.list_filenames()

# Extract and sort all unique values in reverse order
all_dates = sorted(set(fn.split('_')[4].replace('.csv', '') for fn in filenames), reverse=True)
all_sources = sorted(set(fn.split('_')[0] for fn in filenames))
#all_types = sorted(set(fn.split('_')[1] for fn in filenames), reverse=True)
#all_periods = sorted(set(fn.split('_')[2] for fn in filenames))
all_rs_windows = sorted(set(fn.split('_')[3].replace('ibd', '') for fn in filenames))

# Calculate the maximum length of filenames to set dropdown width
if filenames:
    max_filename_length = max(len(fn) for fn in filenames)
else:
    max_filename_length = 0
dropdown_width = f'{max_filename_length * 10}px'  # 10px width per character

# Create dropdowns with dynamic width
date_dropdown = widgets.Dropdown(
    options=all_dates,
    description='Date:',
    layout=widgets.Layout(width=dropdown_width)
)

source_dropdown = widgets.Dropdown(
    options=all_sources,
    description='Source:',
    layout=widgets.Layout(width=dropdown_width)
)

rs_window_dropdown = widgets.Dropdown(
    options=all_rs_windows,
    description='RS window:',
    layout=widgets.Layout(width=dropdown_width)
)

# Create file selection dropdown
file_dropdown = widgets.Dropdown(
    options=[],
    description='File:',
    layout=widgets.Layout(width=dropdown_width)
)

output = widgets.Output()

def update_dropdowns(*args):
    # Filter files based on selected date
    date_filtered_files = [fn for fn in filenames if date_dropdown.value in fn]

    # Update Source dropdown
    available_sources = sorted(set(fn.split('_')[0] for fn in date_filtered_files))
    source_dropdown.options = available_sources
    if source_dropdown.value not in available_sources:
        source_dropdown.value = available_sources[0] if available_sources else None

    # Update RS window dropdown
    available_rs_windows = sorted(set(fn.split('_')[3].replace('ibd', '') for fn in date_filtered_files))
    rs_window_dropdown.options = available_rs_windows
    if rs_window_dropdown.value not in available_rs_windows:
        rs_window_dropdown.value = available_rs_windows[0] if available_rs_windows else None

    # Update file options
    update_file_options()

def update_file_options(*args):
    # Filter files based on selected date, source, and RS window
    filtered_files = [
        fn for fn in filenames
        if (date_dropdown.value in fn and
            source_dropdown.value == fn.split('_')[0] and
            rs_window_dropdown.value == fn.split('_')[3].replace('ibd', ''))
    ]
    file_dropdown.options = filtered_files
    if filtered_files:
        global selected_file, metadata
        file_dropdown.value = filtered_files[0]  # Set initial value to the first match
        selected_file = file_dropdown.value
        metadata = parse_metadata(selected_file)
        with output:
            output.clear_output()
            print_metadata(metadata)
    else:
        file_dropdown.value = None

# Bind event handlers
date_dropdown.observe(update_dropdowns, 'value')
source_dropdown.observe(update_file_options, 'value')
rs_window_dropdown.observe(update_file_options, 'value')

# Display all dropdowns
display(date_dropdown, source_dropdown, rs_window_dropdown, file_dropdown, output)

# Initialize dropdowns
update_dropdowns()

Dropdown(description='Date:', layout=Layout(width='400px'), options=('20241011',), value='20241011')

Dropdown(description='Source:', layout=Layout(width='400px'), options=('TWSE+TPEX', 'U.S.Listed'), value='TWSE…

Dropdown(description='RS window:', layout=Layout(width='400px'), options=('3mo',), value='3mo')

Dropdown(description='File:', layout=Layout(width='400px'), options=(), value=None)

Output()

In [39]:
# @title Step 1.2 Load and Preview the Files
print(f'{selected_file}:')
with requests_cache.disabled():
    df_rs = github.download_csv(selected_file)
display(df_rs)

with requests_cache.disabled():
    df_fin, fn_fin = fin_download_latest_csv(metadata['Source'])
print(f'\n{fn_fin}:')
columns_to_keep = ['Ticker', 'EPS RS', 'TTM EPS', 'Rev RS', 'TTM RPS', 'TTM PE',
                   'Rating (EPS RS)', 'Rating (Rev RS)']
display(df_fin)

print(f'\nMerged:')
df_fin = df_fin[columns_to_keep]
df_merge = pd.merge(df_rs, df_fin, on='Ticker', how='left')
display(df_merge)

U.S.Listed_stocks_2y_ibd3mo_20241011.csv:


,Ticker,Sector,Industry,RS,1wk:end max,1mo:1wk max,3mo:1mo max,6mo:3mo max,9mo:6mo max,Rating (RS),Rating (3M:1M),Rating (6M:3M),Rating (9M:6M),Price,52W pos,MA50,MA200,Volume / VMA50
0,TECX,Healthcare,Biotechnology,20219.48,42663.21,229331.21,153281.31,1476.08,96.15,99,99,99,28,31.00,0.94,20.99,17.22,0.56
1,MIRA,Healthcare,Drug Manufacturers—General,6892.44,6892.44,5947.05,3151.57,99.45,84.47,99,99,44,7,1.08,0.12,1.59,1.17,0.11
2,ASNS,Technology,Communication Equipment,1070.68,1611.94,9728.32,17028.44,1443.03,93.90,99,99,99,18,1.40,0.29,1.48,1.23,0.06
3,RGC,Healthcare,Drug Manufacturers—Specialty & Generic,944.85,944.85,870.96,587.13,88.86,75.46,99,99,12,4,6.90,0.26,7.10,6.31,5.63
4,GNLN,Consumer Defensive,Tobacco,739.47,739.47,619.99,300.56,125.12,85.54,99,98,83,8,4.45,0.16,5.72,5.34,0.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5426,NDRA,Healthcare,Diagnostics & Research,14.05,15.15,23.80,40.96,70.14,123.39,1,1,3,82,0.22,0.00,0.99,23.65,0.35
5427,CDT,Healthcare,Biotechnology,14.00,14.00,11.98,61.73,125.10,242.67,1,3,83,97,0.10,0.00,0.14,2.04,0.37
5428,CNSP,Healthcare,Biotechnology,10.20,10.20,9.40,28.28,55.30,129.39,1,1,1,85,0.12,0.00,0.15,11.61,0.27
5429,SYTA,Technology,Communication Equipment,9.68,10.11,13.39,54.03,81.38,72.05,1,2,6,3,0.83,0.00,1.70,35.52,0.05



U.S.Listed_stocks_fin_20241010.csv:


,Ticker,Sector,Industry,Price,EPS QoQ (%),QoQ 2Q Algo (%),QoQ 3Q Algo (%),EPS YoY (%),YoY 2Q Algo (%),EPS RS,TTM EPS,Rev RS,TTM RPS,TTM PE,Rating (EPS RS),Rating (Rev RS)
0,LUNR,Industrials,Aerospace & Defense,7.1200,9.18,-7.84,-3.79,-4.58,NaN,6.003049e+09,-0.17,83.11,4.810,NaN,99.0,91.0
1,BZFD,Communication Services,Internet Content & Information,2.6900,3.84,-9.17,4.00,2.88,NaN,4.060600e+09,-1.12,-36.62,6.229,NaN,99.0,9.0
2,CART,Consumer Cyclical,Internet Retail,43.9900,-1.14,22.69,1.05,22000000.00,47000000.0,2.300000e+09,-5.56,7.22,13.986,NaN,99.0,76.0
3,ACDC,Energy,Oil & Gas Equipment & Services,7.0900,-37.89,55.43,-1.92,-19.82,NaN,1.882714e+09,-1.43,5.76,13.919,NaN,99.0,74.0
4,KNTK,Energy,Oil & Gas Midstream,48.7800,3.50,-13.17,7.10,0.32,NaN,1.644445e+09,2.83,18.60,24.316,17.17,99.0,83.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5437,ZJYL,Healthcare,Medical Instruments & Supplies,2.5550,NaN,NaN,NaN,NaN,NaN,NaN,0.02,NaN,0.129,127.00,NaN,NaN
5438,ZK,Consumer Cyclical,Auto Manufacturers,25.1500,-0.17,-0.35,0.61,NaN,NaN,NaN,-4.99,NaN,314.047,NaN,NaN,NaN
5439,ZKIN,Basic Materials,Steel,0.5050,NaN,NaN,NaN,NaN,NaN,NaN,-1.95,NaN,3.592,NaN,NaN,NaN
5440,ZOOZ,Consumer Cyclical,Specialty Retail,2.3400,NaN,NaN,NaN,NaN,NaN,NaN,-22.44,NaN,0.069,NaN,NaN,NaN



Merged:


,Ticker,Sector,Industry,RS,1wk:end max,1mo:1wk max,3mo:1mo max,6mo:3mo max,9mo:6mo max,Rating (RS),...,MA50,MA200,Volume / VMA50,EPS RS,TTM EPS,Rev RS,TTM RPS,TTM PE,Rating (EPS RS),Rating (Rev RS)
0,TECX,Healthcare,Biotechnology,20219.48,42663.21,229331.21,153281.31,1476.08,96.15,99,...,20.99,17.22,0.56,NaN,-3.07,NaN,NaN,NaN,NaN,NaN
1,MIRA,Healthcare,Drug Manufacturers—General,6892.44,6892.44,5947.05,3151.57,99.45,84.47,99,...,1.59,1.17,0.11,126.96,-0.86,-1.191000e+01,NaN,NaN,25.0,30.0
2,ASNS,Technology,Communication Equipment,1070.68,1611.94,9728.32,17028.44,1443.03,93.90,99,...,1.48,1.23,0.06,2352.19,-1.40,3.606000e+01,1.772,NaN,96.0,88.0
3,RGC,Healthcare,Drug Manufacturers—Specialty & Generic,944.85,944.85,870.96,587.13,88.86,75.46,99,...,7.10,6.31,5.63,NaN,-0.36,NaN,NaN,NaN,NaN,NaN
4,GNLN,Consumer Defensive,Tobacco,739.47,739.47,619.99,300.56,125.12,85.54,99,...,5.72,5.34,0.03,2176.56,18.78,-4.508900e+02,61.566,0.23,96.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5426,NDRA,Healthcare,Diagnostics & Research,14.05,15.15,23.80,40.96,70.14,123.39,1,...,0.99,23.65,0.35,441.12,-32.00,-1.191000e+01,NaN,NaN,85.0,30.0
5427,CDT,Healthcare,Biotechnology,14.00,14.00,11.98,61.73,125.10,242.67,1,...,0.14,2.04,0.37,176.46,-0.08,-1.191000e+01,NaN,NaN,43.0,30.0
5428,CNSP,Healthcare,Biotechnology,10.20,10.20,9.40,28.28,55.30,129.39,1,...,0.15,11.61,0.27,580.15,-367.38,-1.191000e+01,NaN,NaN,89.0,30.0
5429,SYTA,Technology,Communication Equipment,9.68,10.11,13.39,54.03,81.38,72.05,1,...,1.70,35.52,0.05,381.16,295.46,-3.674000e+01,175.111,0.00,82.0,9.0


## Step 2. Filter Stocks

In [51]:
# @title Filter 1. Sorting {"run":"auto"}
cond0 = "(df['RS'] > 100)" # @param ["(df['RS'] > 100)"]
cond1 = "& (df['Rating (RS)'] > 95)" # @param ["& (df['Rating (RS)'] > 95)", "& (df['Rating (RS)'] > 90)", "& (df['Rating (RS)'] > 85)", "& (df['Rating (RS)'] > 80)", ""]
cond2 = "& ((df['EPS RS'] > 0) | (df['Rev RS'] > 0))" # @param ["& ((df['EPS RS'] > 0) | (df['Rev RS'] > 0))", ""]
cond3 = "& (df['EPS RS'] > 0)" # @param ["& (df['EPS RS'] > 0)", "& (df['EPS RS'] > 50)", "& (df['EPS RS'] > 100)", "& (df['EPS RS'] > 200)", "& (df['EPS RS'] > 500)", ""]
cond4 = "& (df['Rev RS'] > 0)" # @param ["& (df['Rev RS'] > 0)", "& (df['Rev RS'] > 5)", "& (df['Rev RS'] > 10)", "& (df['Rev RS'] > 20)", "& (df['Rev RS'] > 50)", "& (df['Rev RS'] > 100)", ""]
cond5 = "& (df['TTM EPS'] > 0)" # @param ["& (df['TTM EPS'] > 0)", "& (df['TTM RPS'] > 0)", "& ((df['TTM EPS'] > 0) & (df['TTM RPS'] > 0))", ""]
cond6 = "" # @param ["& (df['Price'] < 50)", "& (df['Price'] < 100)", "& (df['Price'] < 200)", ""]
sorted_column = "RS" # @param ["RS","1 Week Ago","1 Month Ago","3 Months Ago","6 Months Ago","Price","MA10","MA30","Volume / VMA10","EPS RS", "Rev RS"]
ascending = False # @param {"type":"boolean"}
num_items = 30 # @param [10, 20, 30] {"type":"raw"}

NUM_CONDS = 7
cond = eval('+'.join([f"cond{i}" for i in range(NUM_CONDS)]))

print_metadata(metadata)
df = df_merge.copy()

df = df[eval(cond)]
df = df.sort_values(by=sorted_column, ascending=ascending)
df_top_f1 = df.head(num_items)
display(df_top_f1)

print_column(df_top_f1, 'Name')
print_column(df_top_f1, 'Ticker')

Source: U.S.Listed
Type: stocks
Period: 2y
RS window: 3mo
Date: 20241011


,Ticker,Sector,Industry,RS,1wk:end max,1mo:1wk max,3mo:1mo max,6mo:3mo max,9mo:6mo max,Rating (RS),...,MA50,MA200,Volume / VMA50,EPS RS,TTM EPS,Rev RS,TTM RPS,TTM PE,Rating (EPS RS),Rating (Rev RS)
52,FFIE,Consumer Cyclical,Auto Manufacturers,214.88,289.83,1015.71,10280.52,2605.81,18.37,98,...,5.19,12.26,2.26,5302.38,866.40,2.824444e+15,0.378,0.00,98.0,97.0
71,SEZL,Financial Services,Credit Services,197.69,197.73,198.04,202.13,386.14,378.67,98,...,136.65,79.94,0.61,1202.40,7.05,4.517000e+01,33.982,22.63,94.0,89.0
94,ADMA,Healthcare,Biotechnology,177.18,178.79,178.84,179.66,159.25,131.29,97,...,17.51,10.16,3.95,514.34,0.14,6.802000e+01,1.447,121.50,87.0,90.0
115,CMPO,Industrials,Metal Fabrication,167.35,167.35,163.61,139.73,122.62,100.22,97,...,11.90,7.68,1.01,158.64,1.04,1.400000e-01,19.384,13.37,34.0,66.0
121,BYU,Financial Services,Shell Companies,164.44,168.56,166.12,147.21,82.60,69.34,97,...,1.74,1.41,6.83,695.39,1.68,3.079000e+01,13.657,0.28,91.0,87.0
124,JVA,Consumer Defensive,Packaged Foods,163.34,164.84,166.25,151.99,141.32,134.52,97,...,2.53,1.76,0.14,1004.75,0.20,1.350000e+00,13.333,14.64,93.0,68.0
160,LWAY,Consumer Defensive,Packaged Foods,148.09,148.09,135.67,112.58,155.92,150.69,96,...,20.25,16.30,0.23,342.89,0.90,1.037000e+01,12.028,29.56,80.0,78.0
163,YOU,Technology,Software—Application,147.68,147.68,144.22,128.12,89.15,100.27,96,...,29.25,21.71,0.48,510.41,0.78,1.811000e+01,7.586,42.67,87.0,83.0
190,CVNA,Consumer Cyclical,Auto & Truck Dealerships,143.62,143.66,145.72,167.39,200.81,165.45,96,...,155.86,104.65,0.61,534.82,2.49,9.650000e+00,101.398,76.91,88.0,78.0
191,UG,Consumer Defensive,Household & Personal Products,143.60,145.43,144.42,131.69,107.16,106.66,96,...,13.61,9.86,1.41,212.56,0.71,6.750000e+00,2.679,17.21,62.0,75.0


FFIE, SEZL, ADMA, CMPO, BYU, JVA, LWAY, YOU, CVNA, UG


In [52]:
# @title Filter 2. Increasing RS > 0 {"run":"auto"}
cond0 = "(df['RS'] > 100)" # @param ["(df['RS'] > 100)"]
cond1 = "" # @param ["& (df['RS'] > df['1wk:end max'])", ""]
cond2 = "& (df['1wk:end max'] > df['1mo:1wk max'])" # @param ["& (df['1wk:end max'] > df['1mo:1wk max'])", "& (df['RS'] > df['1mo:1wk max'])", ""]
cond3 = "& (df['1mo:1wk max'] > df['3mo:1mo max'])" # @param ["& (df['1mo:1wk max'] > df['3mo:1mo max'])", ""]
cond4 = "& (df['3mo:1mo max'] > df['6mo:3mo max'])" # @param ["& (df['3mo:1mo max'] > df['6mo:3mo max'])", ""]
cond5 = "& (df['6mo:3mo max'] > df['9mo:6mo max'])" # @param ["& (df['6mo:3mo max'] > df['9mo:6mo max'])", ""]
cond6 = "& (df['Rating (RS)'] > 90)" # @param ["& (df['Rating (RS)'] > 95)", "& (df['Rating (RS)'] > 90)", "& (df['Rating (RS)'] > 85)", "& (df['Rating (RS)'] > 80)", ""]
cond7 = "& ((df['EPS RS'] > 0) | (df['Rev RS'] > 0))" # @param ["& ((df['EPS RS'] > 0) | (df['Rev RS'] > 0))", ""]
cond8 = "& (df['EPS RS'] > 0)" # @param ["& (df['EPS RS'] > 0)", "& (df['EPS RS'] > 50)", "& (df['EPS RS'] > 100)", "& (df['EPS RS'] > 200)", "& (df['EPS RS'] > 500)", ""]
cond9 = "& (df['Rev RS'] > 0)" # @param ["& (df['Rev RS'] > 0)", "& (df['Rev RS'] > 5)", "& (df['Rev RS'] > 10)", "& (df['Rev RS'] > 20)", "& (df['Rev RS'] > 50)", "& (df['Rev RS'] > 100)", ""]
cond10 = "& ((df['TTM EPS'] > 0) & (df['TTM RPS'] > 0))" # @param ["& (df['TTM EPS'] > 0)", "& (df['TTM RPS'] > 0)", "& ((df['TTM EPS'] > 0) & (df['TTM RPS'] > 0))", ""]
cond11 = "& (df['Price'] < 100)" # @param ["& (df['Price'] < 50)", "& (df['Price'] < 100)", "& (df['Price'] < 200)", ""]
num_items = 20 # @param [10, 20, 30] {"type":"raw"}

NUM_CONDS = 12
cond = eval('+'.join([f"cond{i}" for i in range(NUM_CONDS)]))

print_metadata(metadata)
df = df_merge.copy()

df = df[eval(cond)]
df_top_f2 = df.head(num_items)
display(df_top_f2)

print_column(df_top_f2, 'Name')
print_column(df_top_f2, 'Ticker')

Source: U.S.Listed
Type: stocks
Period: 2y
RS window: 3mo
Date: 20241011


,Ticker,Sector,Industry,RS,1wk:end max,1mo:1wk max,3mo:1mo max,6mo:3mo max,9mo:6mo max,Rating (RS),...,MA50,MA200,Volume / VMA50,EPS RS,TTM EPS,Rev RS,TTM RPS,TTM PE,Rating (EPS RS),Rating (Rev RS)
115,CMPO,Industrials,Metal Fabrication,167.35,167.35,163.61,139.73,122.62,100.22,97,...,11.90,7.68,1.01,158.64,1.04,0.14,19.384,13.37,34.0,66.0
121,BYU,Financial Services,Shell Companies,164.44,168.56,166.12,147.21,82.60,69.34,97,...,1.74,1.41,6.83,695.39,1.68,30.79,13.657,0.28,91.0,87.0
191,UG,Consumer Defensive,Household & Personal Products,143.60,145.43,144.42,131.69,107.16,106.66,96,...,13.61,9.86,1.41,212.56,0.71,6.75,2.679,17.21,62.0,75.0
226,SEI,Energy,Oil & Gas Equipment & Services,138.82,141.58,139.76,127.80,105.85,89.23,95,...,12.36,9.66,1.00,625.00,0.66,1.36,9.563,20.55,90.0,68.0
327,VIRT,Financial Services,Capital Markets,129.41,129.41,127.29,122.78,114.14,102.68,93,...,30.49,23.44,0.84,382.83,1.99,2.82,20.318,16.15,82.0,70.0
334,PPIH,Industrials,Building Products & Equipment,128.87,128.87,122.58,108.81,105.41,103.63,93,...,10.92,9.03,0.30,353.33,1.90,0.61,19.878,6.96,81.0,66.0
339,HNNA,Financial Services,Asset Management,128.74,128.74,127.93,119.29,99.78,94.36,93,...,9.70,7.73,0.22,198.05,0.80,8.47,3.543,12.09,57.0,77.0
423,PAYO,Technology,Software—Infrastructure,124.81,124.81,121.73,112.45,108.41,89.03,91,...,7.14,5.68,1.76,327.18,0.26,10.07,2.511,30.29,79.0,78.0


CMPO, BYU, UG, SEI, VIRT, PPIH, HNNA, PAYO


In [68]:
# @title Filter 3. RS Breakout {"run":"auto"}
base = 100 # @param [100, 130, 160, 200] {"type":"raw"}
cond0 = "(df['RS'] > base)" # @param ["(df['RS'] > base)"]
cond1 = "" # @param ["& (df['1wk:end max'] < base)",""]
cond2 = "" # @param ["& (df['1mo:1wk max'] < base)",""]
cond3 = "& (df['3mo:1mo max'] < base)" # @param ["& (df['3mo:1mo max'] < base)",""]
cond4 = "& (df['6mo:3mo max'] < base)" # @param ["& (df['6mo:3mo max'] < base)",""]
cond5 = "" # @param ["& (df['9mo:6mo max'] < base)",""]
cond6 = "" # @param ["& (df['Rating (RS)'] > 95)", "& (df['Rating (RS)'] > 90)", "& (df['Rating (RS)'] > 85)", "& (df['Rating (RS)'] > 80)", ""]
cond7 = "& (df['Volume / VMA50'] > 2)" # @param ["& (df['Volume / VMA50'] > 1.5)", "& (df['Volume / VMA50'] > 2)", "& (df['Volume / VMA50'] > 3)", "& (df['Volume / VMA50'] > 4)", ""]
cond8 = "" # @param ["& (df['Price'] > df['MA50'])", "& (df['Price'] > df['MA200'])", ""]
cond9 = "" # @param ["& (df['MA50'] > df['MA200'])",""]
cond10 = "" # @param ["& ((df['EPS RS'] > 0) | (df['Rev RS'] > 0))", ""]
cond11 = "" # @param ["& (df['EPS RS'] > 0)", "& (df['EPS RS'] > 50)", "& (df['EPS RS'] > 100)", "& (df['EPS RS'] > 200)", "& (df['EPS RS'] > 500)", ""]
cond12 = "& ((df['TTM EPS'] > 0) & (df['TTM RPS'] > 0))" # @param ["& (df['TTM EPS'] > 0)", "& (df['TTM RPS'] > 0)", "& ((df['TTM EPS'] > 0) & (df['TTM RPS'] > 0))", ""]
cond13 = "" # @param ["& (df['Rev RS'] > 0)", "& (df['Rev RS'] > 5)", "& (df['Rev RS'] > 10)", "& (df['Rev RS'] > 20)", "& (df['Rev RS'] > 50)", "& (df['Rev RS'] > 100)", ""]
cond14 = "" # @param ["& (df['Price'] < 50)", "& (df['Price'] < 100)", "& (df['Price'] < 200)", "& (df['Price'] < 500)", ""]
num_items = 10 # @param [10, 20, 30] {"type":"raw"}

NUM_CONDS = 15
cond = eval('+'.join([f"cond{i}" for i in range(NUM_CONDS)]))

print_metadata(metadata)
df = df_merge.copy()

df = df[eval(cond)]
df_top_f3 = df.head(num_items)
display(df_top_f3)

print_column(df_top_f3, 'Name')
print_column(df_top_f3, 'Ticker')

Source: TWSE+TPEX
Type: stocks
Period: 1y
RS window: 3mo
Date: 20241011


,Ticker,Sector,Industry,RS,1wk:end max,1mo:1wk max,3mo:1mo max,6mo:3mo max,9mo:6mo max,Rating (RS),...,MA50,MA200,Volume / VMA50,EPS RS,TTM EPS,Rev RS,TTM RPS,TTM PE,Rating (EPS RS),Rating (Rev RS)
1302,CCK,Consumer Cyclical,Packaging & Containers,109.13,109.13,106.98,99.17,95.75,97.57,76,...,90.63,83.52,3.51,176.85,3.61,-10.40,98.285,25.38,43.0,39.0
1669,AUBN,Financial Services,Banks—Regional,106.07,106.07,104.59,94.98,87.89,92.96,69,...,19.77,19.44,3.63,98.80,0.18,-15.65,6.359,115.17,22.0,19.0
1997,OVBC,Financial Services,Banks—Regional,103.61,103.61,102.58,94.83,95.13,94.07,63,...,23.99,23.42,2.08,169.29,2.36,-8.37,11.719,9.92,38.0,44.0
2070,TD,Financial Services,Banks—Diversified,103.16,103.20,102.23,96.31,90.26,97.04,62,...,60.60,58.98,5.09,-236.90,3.17,-3.62,29.497,18.78,9.0,57.0
2282,ALCO,Consumer Defensive,Farm Products,101.48,101.67,101.48,97.31,93.95,107.44,58,...,28.02,27.75,3.88,-168.43,3.41,31.38,6.089,7.87,10.0,87.0


CCK, AUBN, OVBC, TD, ALCO


In [69]:
# @title Filter 4. Groupby "Industry" {"run":"auto"}
cond0 = "(df['RS'] > 100)" # @param ["(df['RS'] > 100)"]
cond1 = "& (df['Rating (RS)'] > 95)" # @param ["& (df['Rating (RS)'] > 95)", "& (df['Rating (RS)'] > 90)", "& (df['Rating (RS)'] > 85)", "& (df['Rating (RS)'] > 80)", ""]
num_items = 2 # @param [1, 2, 3, 4, 5] {"type":"raw"}

NUM_CONDS = 2

from vistock.ranking_utils import append_ratings, groupby_industry

print_metadata(metadata)
stock_df = df_merge.copy()

# Filter out rows with NaN in the 'Ticker' column
#stock_df = stock_df[stock_df['Ticker'].notna()]

rs_columns = ['RS',
              '1mo:1wk max', '3mo:1mo max', '6mo:3mo max', '9mo:6mo max']
if 'Name' in stock_df.columns:
    columns = ['Sector', 'Ticker', 'Name'] + rs_columns
else:
    columns = ['Sector', 'Ticker'] + rs_columns
df = groupby_industry(stock_df, columns, key='RS')

df = df.sort_values(by='RS', ascending=False).reset_index(drop=True)
rating_columns = ['Rating (RS)', 'Rating (1M:1W)', 'Rating (3M:1M)',
                  'Rating (6M:3M)', 'Rating (9M:6M)']
df = append_ratings(df, rs_columns, rating_columns)

cond = eval('+'.join([f"cond{i}" for i in range(NUM_CONDS)]))
df = df[eval(cond)]
df_top_f4 = df.head(num_items)
display(df_top_f4)

print_column(df_top_f4, 'Name')
print_column(df_top_f4, 'Ticker')

Source: U.S.Listed
Type: stocks
Period: 2y
RS window: 3mo
Date: 20241011


,Industry,Sector,Ticker,RS,1mo:1wk max,3mo:1mo max,6mo:3mo max,9mo:6mo max,Rating (RS),Rating (1M:1W),Rating (3M:1M),Rating (6M:3M),Rating (9M:6M)
0,Drug Manufacturers—General,Healthcare,"MIRA,GRFS,GILD,BMY,ABBV,SNY,NVS,LLY,JNJ,AMGN,A...",477.39,428.99,274.61,101.39,100.97,99,98,96,22,21
1,Tobacco,Consumer Defensive,"GNLN,KAVL,TPB,PM,BTI,MO,UVV,ISPR,RLX,XXII",174.39,184.43,163.55,139.23,95.46,98,96,94,92,8


MIRA,GRFS,GILD,BMY,ABBV,SNY,NVS,LLY,JNJ,AMGN,AZN,PFE,OGN,GSK,MRK,BIIB,SCLX,AMRN, GNLN,KAVL,TPB,PM,BTI,MO,UVV,ISPR,RLX,XXII


## Step 3. Visualize Filtered Stocks

In [ ]:
# @title Plot 1. IBD Relative Strength Comparison {"run":"auto"}
source = "Filter 4. Groupby Industry" # @param ["Filter 1. Sorting", "Filter 2. Increasing RS","Filter 3. RS Breakout", "Filter 4. Groupby Industry", "Major Global Stock Indices", "Sector Indices"]
backend = "mplfinance" # @param ["mplfinance","Plotly"]

import matplotlib.pyplot as plt
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display, clear_output

MAX_STOCK_SELECTION = 10
N_COLS = 8  # the number of columns of the grid layout for checkboxes

symbols = {
    'Filter 1. Sorting': lambda: tickers_from_df(df_top_f1),
    'Filter 2. Increasing RS': lambda: tickers_from_df(df_top_f2),
    'Filter 3. RS Breakout': lambda: tickers_from_df(df_top_f3),
    'Filter 4. Groupby Industry': lambda: tickers_from_df(df_top_f4),
    'Major Global Stock Indices': major_indices,
    'Sector Indices': sector_indices,
}[source]()

checkboxes = cbs_create(symbols, MAX_STOCK_SELECTION)
checkbox_grid = cbs_with_grid(checkboxes, N_COLS)
btn_unselect_all = widgets.Button(description="Unselect All")
btn_unselect_all.on_click(lambda b: cbs_unselect_all(checkboxes))
btn_select_top = widgets.Button(description="Select Top 10")
btn_select_top.on_click(lambda b: cbs_select_top(checkboxes, MAX_STOCK_SELECTION))

period_dropdown = create_period_dropdown(metadata['Period'])
interval_dropown = create_interval_dropdown('1d')
rs_window_dropdown = create_rs_window_dropdown(metadata['RS window'])

cmp_theme_dropdown = {
    'mplfinance': create_style_dropdown('Comparison Theme:', 'charles'),
    'Plotly': create_template_dropdown('Comparison Theme:', 'plotly_dark'),
}[backend]
btn_plot = widgets.Button(description="Generate Plot")

out_msg, out_fig = widgets.Output(), widgets.Output()

ui = widgets.VBox([
    checkbox_grid,
    widgets.HBox([btn_unselect_all, btn_select_top]),
    widgets.VBox([period_dropdown, interval_dropown, rs_window_dropdown]),
    widgets.VBox([cmp_theme_dropdown, btn_plot]),
    out_msg, out_fig
])
display(ui)

import vistock.mpl as mpl
import vistock.plotly as ply

rs_cmp = {
    'mplfinance': mpl.ibd_rs_cmp,
    'Plotly': ply.ibd_rs_cmp,
}[backend]

def on_checkbox_change(change):
    selected_count = sum([cb.value for cb in checkboxes])
    if selected_count > MAX_STOCK_SELECTION:
        # Uncheck the last checked box if selection exceeds limit
        changed_checkbox = change['owner']
        changed_checkbox.value = False
        with out_msg:
            out_fig.clear_output()
            print(f"Only {MAX_STOCK_SELECTION} stocks can be selected at most.")

# Bind the checkbox change event to the function
for checkbox in checkboxes:
    checkbox.observe(on_checkbox_change, names='value')

def on_plot_click(b):
    symbols = cbs_get_selected(checkboxes)
    if not symbols:
        with out_msg:
            out_fig.clear_output()
            print("No stocks selected. Please select at least one stock.")
        return
    with out_fig:
        out_msg.clear_output()
        clear_output()
        interval = interval_dropown.value
        period = period_dropdown.value
        rs_window = rs_window_dropdown.value
        if rs_cmp is mpl.ibd_rs_cmp:
            rs_cmp.plot(symbols, interval=interval, period=period,
                        rs_window=rs_window,
                        style=cmp_theme_dropdown.value,
                        color_cycle=plt.cm.Paired.colors)
        else: # Plotly
            rs_cmp.plot(symbols, interval=interval, period=period,
                        rs_window=rs_window,
                        template=cmp_theme_dropdown.value,
                        colorway=px.colors.qualitative.Set3)

btn_plot.on_click(on_plot_click)

if backend == 'Plotly':
    enable_plotly_in_cell()


In [ ]:
# @title Plot 2. IBD Stock Chart {"run":"auto"}
source = "Filter 4. Groupby Industry" # @param ["Filter 1. Sorting", "Filter 2. Increasing RS","Filter 3. RS Breakout","Filter 4. Groupby Industry","Major Global Stock Indices", "U.S. Listed Stocks", "Taiwan Stocks"]
backend = "mplfinance" # @param ["mplfinance","Plotly"]

import functools as ft
import ipywidgets as widgets
from IPython.display import display, clear_output
import yfinance as yf

import vistock.stock_indices as si
import vistock.tw as tw

symbols = {
    'Filter 1. Sorting': lambda: tickers_from_df(df_top_f1),
    'Filter 2. Increasing RS': lambda: tickers_from_df(df_top_f2),
    'Filter 3. RS Breakout': lambda: tickers_from_df(df_top_f3),
    'Filter 4. Groupby Industry': lambda: tickers_from_df(df_top_f4),
    'Major Global Stock Indices': major_indices,
    'U.S. Listed Stocks': ft.partial(si.get_tickers, 'USLS'),
    'Taiwan Stocks': ft.partial(si.get_tickers, 'TWSE+TPEX+ESB'),
}[source]()
if source == 'Taiwan Stocks':
    symbols = [f"{tw_stock_name(s)} {s}" for s in symbols]

dropdowns, layout = create_search_dropdowns(symbols, 1)

period_dropdown2 = create_period_dropdown(metadata['Period'])
interval_dropown2 = create_interval_dropdown('1d')
rs_window_dropdown2 = create_rs_window_dropdown(metadata['RS window'])

stock_theme_dropdown = {
    'mplfinance': create_style_dropdown('Stock Theme:', 'yahoo'),
    'Plotly': create_template_dropdown('Stock Theme:', 'plotly'),
}[backend]

btn_plot_prc = widgets.Button(description="Price/RS/Volume Chart",
                              layout=widgets.Layout(width='168px'))
btn_plot_fin = widgets.Button(description="Financial Chart",
                              layout=widgets.Layout(width='168px'))
btn_report_q = widgets.Button(description="Quarterly Report",
                              layout=widgets.Layout(width='168px'))
btn_report_a = widgets.Button(description="Annual Report",
                              layout=widgets.Layout(width='168px'))
btn_clear_last = widgets.Button(description="Clear Last")
btn_clear_all = widgets.Button(description="Clear All")
#outputs = widgets.VBox()

ui = widgets.VBox([
    layout,
    period_dropdown2, interval_dropown2, rs_window_dropdown2, stock_theme_dropdown,
    widgets.HBox([btn_plot_prc, btn_plot_fin]),
    widgets.HBox([btn_report_q, btn_report_a]),
    widgets.HBox([btn_clear_last, btn_clear_all]),
    outputs
])
display(ui)

import vistock.mpl as mpl
import vistock.plotly as ply

stock_chart = {
    'mplfinance': mpl.ibd_rs,
    'Plotly': ply.ibd_rs,
}[backend]

def get_symbols():
    symbols = get_dropdowns_selected_options(dropdowns)
    if source == 'Taiwan Stocks':
        symbols = [s.split()[0] for s in symbols]
    return symbols

def on_plot_prc_click(b):
    selected = get_symbols()
    new_output = widgets.Output()
    with new_output:
        interval = interval_dropown2.value
        period = period_dropdown2.value
        rs_window = rs_window_dropdown2.value
        if not selected:
            print("No Stock Selected!")
        elif stock_chart is mpl.ibd_rs:
            stock_chart.plot(selected[0], interval=interval, period=period,
                             rs_window=rs_window,
                             style=stock_theme_dropdown.value,
                             legend_loc='upper left')
        else: # Plotly
            stock_chart.plot(selected[0], interval=interval, period=period,
                             rs_window=rs_window,
                             template=stock_theme_dropdown.value)
    outputs.children = (new_output,) + outputs.children

import vistock.mpl.financials as mpl_fin
import vistock.plotly.financials as ply_fin

fin_chart = {
    'mplfinance': mpl_fin,
    'Plotly': ply_fin,
}[backend]

def on_plot_fin_click(b):
    selected = get_symbols()
    new_output = widgets.Output()
    with new_output:
        if not selected:
            print("No Stock Selected!")
        elif fin_chart is mpl_fin:
            fin_chart.plot(selected[0], style=stock_theme_dropdown.value)
        else: # Plotly
            fin_chart.plot(selected[0], template=stock_theme_dropdown.value)
    outputs.children = (new_output,) + outputs.children

def on_report_q_click(b):
    selected = get_symbols()
    new_output = widgets.Output()
    with new_output:
        if not selected:
            print("No Stock Selected!")
        else:
            symbol = tw.as_yfinance(selected[0])
            ticker = tw.as_yfinance(symbol)
            print(f"\n{symbol} Quarterly Financials:")
            display(yf.Ticker(ticker).quarterly_financials)
    outputs.children = (new_output,) + outputs.children

def on_report_a_click(b):
    selected = get_symbols()
    new_output = widgets.Output()
    with new_output:
        if not selected:
            print("No Stock Selected!")
        else:
            symbol = tw.as_yfinance(selected[0])
            ticker = tw.as_yfinance(symbol)
            print(f"\n{symbol} Annual Financials:")
            display(yf.Ticker(ticker).financials)
    outputs.children = (new_output,) + outputs.children

def on_clear_last_click(b):
    if outputs.children:
        children = list(outputs.children)
        children.pop(0)
        outputs.children = tuple(children)

def on_clear_all_click(b):
    outputs.children = ()

btn_plot_prc.on_click(on_plot_prc_click)
btn_plot_fin.on_click(on_plot_fin_click)
btn_report_q.on_click(on_report_q_click)
btn_report_a.on_click(on_report_a_click)
btn_clear_last.on_click(on_clear_last_click)
btn_clear_all.on_click(on_clear_all_click)

if backend == 'Plotly':
    enable_plotly_in_cell()
